In [28]:
import pandas as pd
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(user='ebouille')
import os
username = os.environ['JUPYTERHUB_USER']
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pickle
import heapq
import pickle
import networkx as nx
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output
from tqdm.notebook import tqdm

from utils.file_utils import *
from utils.graph_utils import *
from utils.delay_utils import *
from utils.viz_utils import *
from utils.plan_utils import *

In [29]:
stops = read_parquet_from_hdfs("stops", username)
trips = read_parquet_from_hdfs("trips", username)
routes = read_parquet_from_hdfs("routes", username)
trip_to_product = trip_to_product_func(username)

direc = "/work/final-assignment-group-y/data/"
filename = "graph_reverse.pickle"
with open(direc + filename, 'rb') as config_dictionary_file:
    G_rev = pickle.load(config_dictionary_file)
    
scaler = read_scaler_from_hdfs("delays_scaler_model", username)
log = read_logistic_regression_from_hdfs("delays_log_model", username)

gt = GraphTraversals(None, G_rev)

In [37]:
def sample_routes(arrival_time, travel_date, confidence_threshold, n):
    res = []
    for _ in tqdm(range(n)):
        try:
            origin = stops.stop_id.sample().iloc[0]
            destination = origin
            while origin == destination:
                destination = stops.stop_id.sample().iloc[0]
            A = plan_travel(gt, origin, destination, arrival_time, travel_date, 
                            1, confidence_threshold, fast=True, 
                            scaler=scaler, model=log, product_id_map=product_id_map, trip_to_product=trip_to_product)
            res.append(A)
        except:
            pass
    return res

In [34]:
arrival_time = "14:00"
travel_date = "14.05.2019"

In [ ]:
A_08 = sample_routes(arrival_time, travel_date, 0.8, 50)

In [ ]:
A_096 = sample_routes(arrival_time, travel_date, 0.96, 50)

In [ ]:
A_base1 = sample_routes(arrival_time, travel_date, None, 50)

In [42]:
with open('../data/A_08.pickle', 'wb') as f:
    pickle.dump(A_08, f)

with open('../data/A_096.pickle', 'wb') as f:
    pickle.dump(A_096, f)
    
with open('../data/A_base1.pickle', 'wb') as f:
    pickle.dump(A_base1, f)